In [3]:
import numpy as np

In [11]:
def getCard():
    card =  np.random.choice(range(1, 14))
    if card == 1:
        return 11
    if card >= 10:
        return 10
    return card

In [5]:
def toss():
    if np.random.random() < 0.5:
        return True
    else:
        return False

In [9]:
dealerSum = 10
takeCard = dealerSum < 17
if not takeCard:
    print("a")

In [26]:
print(5) 

5


In [27]:
def fetchCardAndUpdate(currentSum, usableAce, debug = False):
    busted = False
    card = getCard()
    if debug: print(card)
    currentSum += card
    if currentSum > 21:
        if usableAce and  card == 11:
                currentSum -= 10
        elif usableAce or card == 11:
                currentSum -= 10
                usableAce = False
        else:
            busted = True
    elif card == 11:
        usableAce = True
     
    return currentSum, usableAce, busted

In [36]:
DEALER_WIN = 0
PLAYER_WIN = 1
PUSH = 2

def play(debug = False):
    playerSum = 0
    dealerSum = 0
    usableAcePlayer = False
    usableAceDealer = False
    playerCard = getCard()
    playerSum += playerCard
    if (playerCard == 11):
        usableAcePlayer = True
    if debug: print("player: ", playerCard)
    
    dealerCard = getCard()
    dealerSum += dealerCard
    if (dealerCard == 11):
        usableAceDealer = True
    
    if debug: print("dealer: ", dealerCard)
    
    playerCard = getCard()
    playerSum += playerCard
    if (playerCard == 11):
        usableAcePlayer = True
    if playerSum == 22:
        playerSum -= 10
    
    if debug: print("player: ", playerCard)
    if debug: print("playerSum: ", playerSum)
    
    trajectory = []
    # player turn
    while True:
        takeCard = False
        if (playerSum <= 11):
            takeCard = True
        elif playerSum == 21:
            takeCard = False
        else:
            takeCard = toss()
            trajectory.append((playerSum, dealerSum, usableAcePlayer,takeCard))
        
        if not takeCard:
            break
         
        if debug: print("player:") 
        playerSum, usableAcePlayer, busted = fetchCardAndUpdate(playerSum, usableAcePlayer)
        if debug: print("playersum: ", playerSum) 
        if busted:
            return DEALER_WIN, trajectory
    while True:
        takeCard = dealerSum < 17
        if not takeCard:
            break
        if debug: print("dealer:")
        dealerSum, usableAceDealer, busted = fetchCardAndUpdate(dealerSum, usableAceDealer)
        if debug: print("dealerSum: ", dealerSum)
        if busted:
            return PLAYER_WIN, trajectory

    if dealerSum > playerSum:
        return DEALER_WIN, trajectory
    elif playerSum > dealerSum:
        return PLAYER_WIN, trajectory
    else:
        return PUSH, trajectory

In [37]:
def monteCarlo(nEpisodes):
    outcomes = np.zeros(3)
    stateActionValues = np.zeros((9, 10, 2, 2))
    stateActionCount = np.ones((9, 10, 2, 2))
    for episode in range(nEpisodes):
        outcome, trajectory = play()
        reward = 0
        if outcome == DEALER_WIN:
            reward = -1
        elif outcome == PLAYER_WIN:
            reward = 1
        else:
            reward = 0
        
        for playerSum, dealerSum, usableAce,  action in trajectory:
            playerSumState = playerSum - 12
            dealerSumState = dealerSum - 2
            usableAceState = 1 if usableAce else 0
            actionState = 1 if action else 0
            stateActionValues[playerSumState, dealerSumState, usableAceState, actionState] += reward
            stateActionCount[playerSumState, dealerSumState, usableAceState, actionState] += 1
        outcomes[outcome] += 1
    return outcomes, stateActionValues/stateActionCount

In [56]:
outcomes, state_values = monteCarlo(10000000)

In [54]:
withAce = np.zeros((9,10))
for i in range(9):
    for j in range(10):
        withAce[i, j] = np.argmax(state_values[i, j, 1, :])

In [55]:
withAce

array([[ 1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  1.,  1.,  1.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])